In [1]:
!pip install graphviz --user
!pip install pyvis --user
!pip install ipython --user


     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------- --------------------- 20.5/47.0 kB 330.3 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/47.0 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 47.0/47.0 kB 391.5 kB/s eta 0:00:00
     ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
      --------------------------------------- 10.2/756.0 kB ? eta -:--:--
     - ----------------------------------- 30.7/756.0 kB 435.7 kB/s eta 0:00:02
     --- --------------------------------- 61.4/756.0 kB 544.7 kB/s eta 0:00:02
     ----- ------------------------------ 122.9/756.0 kB 804.6 kB/s eta 0:00:01
     -------- --------------------------- 174.1/756.0 kB 876.1 kB/s eta 0:00:01
     ------------ ------------------------- 245.8/756.0 kB 1.0 MB/s eta 0:00:01
     ------------- ---------------------- 286.7/756.0 kB 983.9 kB/s eta 0:00:01
     ----------------- -------------------- 348.2/756.0 kB 1

In [1]:
import pandas as pd
import numpy as np
from graphviz import Digraph #install graphviz from pypi.org, or run pip install with administrator rights
import networkx as nx #install networkx
import warnings
warnings.filterwarnings("ignore")
from pyvis.network import Network #install pyvis
from IPython.core.display import display, HTML
from IPython.display import Javascript as JS

In [2]:
transaction_data = pd.read_csv('prepared_data.csv', delimiter= ';') 
#synthetic DF (in the original, if we load from CFT, we get an upload of transactions containing the date,
#account number of the sender, recipient, amount and date,
#next, separately upload the directory tax ID + organization name and add them into the Payer, Recipient field, implemented in an SQL query)

In [3]:
df_f = pd.read_csv('prepared_data.csv', delimiter= ';') 

In [4]:
clients = pd.concat([transaction_data['Payer'], transaction_data['Recipient']]).unique()

# We divide clients into two groups with a ratio of 30/70
group1_size = int(0.3 * len(clients))
group1_clients = np.random.choice(clients, size=group1_size, replace=False)
group2_clients = np.setdiff1d(clients, group1_clients)

# Create columns "flag1" and "flag2" and fill them with zeros
transaction_data['flag1'] = 0
transaction_data['flag2'] = 0

# Fill the "flag1" column with units for clients from the first group
transaction_data.loc[(transaction_data['Payer'].isin(group1_clients)) | (transaction_data['Recipient'].isin(group1_clients)), 'flag1'] = 1

# Fill the "flag2" column with units for clients from the second group
transaction_data.loc[(transaction_data['Payer'].isin(group2_clients)) | (transaction_data['Recipient'].isin(group2_clients)), 'flag2'] = 1

In [5]:
transaction_data['payer-buyer'] = transaction_data['Payer']+'-->'+ transaction_data['Recipient']

In [6]:
vacalabry = transaction_data[['payer-buyer', 'flag1', 'flag2']]
vacalabry = vacalabry.drop_duplicates()

In [7]:
vacalabry # legend reference 0 - is not a borrower, 1 - is a borrower

,payer-buyer,flag1,flag2
0,Client_171-->Client_73,0,1
1,Client_74-->Client_171,0,1
2,Client_180-->Client_171,0,1
3,Client_73-->Client_171,0,1
4,Client_16-->Client_171,0,1
...,...,...,...
694,Client_72-->Client_6,1,1
698,Client_72-->Client_32,1,1
702,Client_81-->Client_72,0,1
707,Client_167-->Client_72,0,1


In [8]:
#Group DF and count the sums and number of connections
df_t_1 = pd.DataFrame(transaction_data['payer-buyer'].value_counts()).reset_index()
df_t_1.columns = ['Transact', 'Frequency',]
df_t_1['Sum_tr'] = df_t_1['Transact'].map(transaction_data.groupby('payer-buyer')['Sum'].agg('sum'))
transact = df_t_1['Transact'].values.tolist()
counts = df_t_1['Frequency'].values.tolist()
summ_tr = df_t_1['Sum_tr'].values.tolist()

In [9]:
#forming the final DF
df = df_t_1['Transact'].str.split('-->',expand=True)
df.columns=['Payer','Recipient']
for column in df.columns:
    df[column] = df[column].str.replace(column+'=','')

final_df = pd.concat([df_t_1,df],axis=1)

In [10]:
final_df

,Transact,Frequency,Sum_tr,Payer,Recipient
0,Client_161-->Client_86,62,450477.68993,Client_161,Client_86
1,Client_22-->Client_22,55,136094.00000,Client_22,Client_22
2,Client_22-->Client_20,40,51100.00000,Client_22,Client_20
3,Client_153-->Client_98,37,22900.00000,Client_153,Client_98
4,Client_22-->Client_4,29,42782.00000,Client_22,Client_4
...,...,...,...,...,...
208,Client_22-->Client_135,1,10000.00000,Client_22,Client_135
209,Client_22-->Client_2,1,19000.00000,Client_22,Client_2
210,Client_22-->Client_191,1,1000.00000,Client_22,Client_191
211,Client_22-->Client_79,1,1000.00000,Client_22,Client_79


In [11]:
final_df_for_graph = final_df.merge(vacalabry, left_on='Transact', right_on='payer-buyer', how='left')
final_df_for_graph = final_df_for_graph.drop(['payer-buyer'],axis=1)

In [12]:
final_df_for_graph

,Transact,Frequency,Sum_tr,Payer,Recipient,flag1,flag2
0,Client_161-->Client_86,62,450477.68993,Client_161,Client_86,1,1
1,Client_22-->Client_22,55,136094.00000,Client_22,Client_22,0,1
2,Client_22-->Client_20,40,51100.00000,Client_22,Client_20,0,1
3,Client_153-->Client_98,37,22900.00000,Client_153,Client_98,1,0
4,Client_22-->Client_4,29,42782.00000,Client_22,Client_4,0,1
...,...,...,...,...,...,...,...
208,Client_22-->Client_135,1,10000.00000,Client_22,Client_135,0,1
209,Client_22-->Client_2,1,19000.00000,Client_22,Client_2,1,1
210,Client_22-->Client_191,1,1000.00000,Client_22,Client_191,0,1
211,Client_22-->Client_79,1,1000.00000,Client_22,Client_79,0,1


In [15]:
got_net = Network(notebook=True, height="900px", width="1800px", bgcolor="#11111", font_color="black", directed=True)
got_net.barnes_hut(gravity=-1000, central_gravity=0, spring_length=250, spring_strength=0.01, damping=0.09, overlap=0)
got_net.set_edge_smooth('continuous')

got_net.set_options("""
    var options = {
        "nodes": {
            "font": {
                "size": 10
            }
        },
        "edges": {
            "font": {
                "size": 10
            }
        }
    }
    """
)

got_data = final_df_for_graph

sources = got_data['Payer']
targets = got_data['Recipient']
weights = got_data['Sum_tr']
F = got_data['flag1']
N = got_data['flag2']
edge_data = zip(sources, targets, weights, N, F)

for src, dst, w, F, N in edge_data:
   # Determine the color of nodes depending on the values of F and N
    if F == 1 and N == 0:
        got_net.add_node(src, src, title=src, color="blue")  # sender nodes
        got_net.add_node(dst, dst, title=dst, color="red")  # recipient nodes
    elif F == 0 and N == 1:
        got_net.add_node(src, src, title=src, color="red")  # sender nodes
        got_net.add_node(dst, dst, title=dst, color="blue") # recipient nodes
    elif F == 0 and N == 0:
        got_net.add_node(src, src, title=src, color="blue") # sender nodes
        got_net.add_node(dst, dst, title=dst, color="blue")  # recipient nodes
    elif F == 1 and N == 1:
        got_net.add_node(src, src, title=src, color="red")  # sender nodes
        got_net.add_node(dst, dst, title=dst, color="red")  # recipient nodes
    got_net.add_edge(src, dst, value=w)  # sender-receiver edges

neighbor_map = got_net.get_adj_list()

for node in got_net.nodes:
    node_id = node["id"]
    node_neighbors = neighbor_map[node_id]
    neighbor_descriptions = []
    for neighbor in node_neighbors:
        link_data = got_data[(got_data['Payer'] == node_id) & (got_data['Recipient'] == neighbor)]
        sum_tr = link_data['Sum_tr'].values[0]
        num_transactions = link_data['Frequency'].values[0]
        neighbor_description = f"{neighbor} (bond amount: {sum_tr} $, number of transactions: {num_transactions})"
        neighbor_descriptions.append(neighbor_description)
    if neighbor_descriptions:
        node["title"] += f"\nBound with:\n" + "\n".join(neighbor_descriptions)
    node["value"] = len(node_neighbors)

for edge in got_net.edges:
    src = edge["from"]
    dst = edge["to"]
    edge_data = got_data[(got_data["Payer"] == src) & (got_data["Recipient"] == dst)]
    num_transactions = edge_data["Frequency"].iloc[0]
    total_amount = edge_data["Sum_tr"].sum()
    edge["title"] = f"Bond amount: {total_amount} $.\nNumber of transactions: {num_transactions}"

df_f = pd.read_csv('prepared_data.csv', delimiter=';')
#got_net.show("Tran_connect_graph_vis.html")
got_net.save_graph("Tran_connect_graph_vis.html")